In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


# CHANGELOG
Compared to version 3:
- [ ] Use Optuna for hyperparameter tuning

In [2]:
train_orig = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])
train_xtra = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv', index_col=['id'])

test_idx = test.index
train = pd.concat([train_orig, train_xtra], axis=0)
train

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
4194313,Nike,Canvas,NaN,3.0,Yes,Yes,Messenger,Blue,28.098120,104.74460
4194314,Puma,Leather,Small,10.0,Yes,Yes,Tote,Blue,17.379531,122.39043
4194315,Jansport,Canvas,Large,10.0,No,No,Backpack,Red,17.037708,148.18470


## Get Ordinal columns and One Hot columns

In [3]:
ord_cols = ['Size', 'Laptop Compartment', 'Waterproof']
oh_cols = ['Brand', 'Material', 'Style', 'Color']

## Transform features

 *Define ordering for each value*

In [4]:
train['Size'] = train['Size'].map({'Small' : 0, 'Medium' : 1, 'Large' : 2})
train['Waterproof'] = train['Waterproof'].map({'No': 0, 'Yes' : 1})
train['Laptop Compartment'] = train['Laptop Compartment'].map({'No' : 0, 'Yes' : 1})

train

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,1.0,7.0,1.0,0.0,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,0.0,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,0.0,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,0.0,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,1.0,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
4194313,Nike,Canvas,NaN,3.0,1.0,1.0,Messenger,Blue,28.098120,104.74460
4194314,Puma,Leather,0.0,10.0,1.0,1.0,Tote,Blue,17.379531,122.39043
4194315,Jansport,Canvas,2.0,10.0,0.0,0.0,Backpack,Red,17.037708,148.18470


In [5]:
test['Size'] = test['Size'].map({'Small' : 0, 'Medium' : 1, 'Large' : 2})
test['Waterproof'] = test['Waterproof'].map({'No': 0, 'Yes' : 1})
test['Laptop Compartment'] = test['Laptop Compartment'].map({'No' : 0, 'Yes' : 1})

test

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,0.0,2.0,0.0,0.0,Tote,Green,20.671147
300001,Nike,Canvas,1.0,7.0,0.0,1.0,Backpack,Green,13.564105
300002,Adidas,Canvas,2.0,9.0,0.0,1.0,Messenger,Blue,11.809799
300003,Adidas,Nylon,2.0,1.0,1.0,0.0,Messenger,Green,18.477036
300004,NaN,Nylon,2.0,2.0,1.0,1.0,Tote,Black,9.907953
...,...,...,...,...,...,...,...,...,...
499995,Adidas,Canvas,2.0,2.0,1.0,0.0,Messenger,Red,7.383498
499996,Nike,Polyester,0.0,9.0,0.0,1.0,Messenger,Pink,6.058394
499997,Jansport,Nylon,0.0,9.0,0.0,1.0,Tote,Green,26.890163


## Create transformers for each type of column

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

X = train.drop(columns = ['Price'])
y = train['Price']

cat_cols = train.columns.drop(['Weight Capacity (kg)', 'Price'])
oh_cols = ['Brand', 'Material', 'Style', 'Color']


# The following is for using XGBoost, which does not take strings
le = LabelEncoder()
for col in oh_cols:
    X[col] = le.fit_transform(X[col])
    test[col] = le.fit_transform(test[col])

categorical_transformer = Pipeline(
    steps = [
        ('impute', SimpleImputer(strategy = 'most_frequent'))
    ]
)

numerical_transformer = Pipeline(
    steps = [
        ('impute', SimpleImputer(strategy = 'mean')),
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cat', categorical_transformer, cat_cols),
        ('oh encode', OneHotEncoder(sparse_output = False), oh_cols),
        ('num', numerical_transformer, ['Weight Capacity (kg)'])
    ]
)

X = preprocessor.fit_transform(X)
X

array([[ 1.        ,  1.        ,  1.        , ...,  0.        ,
         0.        , -0.91771889],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.30057552],
       [ 4.        ,  1.        ,  0.        , ...,  1.        ,
         0.        , -0.19600981],
       ...,
       [ 1.        ,  0.        ,  2.        , ...,  1.        ,
         0.        , -0.13950869],
       [ 3.        ,  0.        ,  1.        , ...,  0.        ,
         0.        ,  1.54508317],
       [ 4.        ,  3.        ,  1.        , ...,  0.        ,
         0.        ,  0.72654453]])

In [7]:
test = preprocessor.fit_transform(test)
test

array([[ 3.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.38419468],
       [ 2.        ,  0.        ,  1.        , ...,  0.        ,
         0.        , -0.63536177],
       [ 0.        ,  0.        ,  2.        , ...,  0.        ,
         0.        , -0.88702955],
       ...,
       [ 1.        ,  2.        ,  0.        , ...,  0.        ,
         0.        ,  1.27635752],
       [ 3.        ,  2.        ,  2.        , ...,  0.        ,
         0.        ,  1.11554046],
       [ 4.        ,  1.        ,  1.        , ...,  0.        ,
         0.        ,  1.60413757]])

In [8]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna

lgb_data = lgb.Dataset(X, label=y)

def objective(trial):
    
    params = {
        'objective': 'regression',  # Regression task
        'metric': 'rmse',          # Root Mean Squared Error
        'device': 'gpu',
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.1),  
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.1, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.5, 1.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 15),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),
        'subsample': trial.suggest_uniform('subsample', 0.7, 0.9),
        'num_leaves': trial.suggest_int('num_leaves', 31, 60),
        'min_split_gain': trial.suggest_uniform('min_split_gain', 0.1, 0.3),
        'feature_pre_filter': False
    }
    
    # Computationally time consuming to run
    
    cv_results = lgb.cv(
        params,                    # Hyperparameters
        lgb_data,                  # Dataset
        num_boost_round=100,       # Number of boosting rounds
        nfold=5,                   # Number of folds for cross-validation
        stratified=False,          # Stratified sampling (not needed for regression)
        shuffle=True,              # Shuffle data before splitting
        metrics='rmse',            # Evaluation metric
        seed=42,                    # Random seed for reproducibility
        callbacks=[
            lgb.early_stopping(stopping_rounds=5), # callbacks introduced in lgbm version 4
        ]
    )

    return cv_results['valid rmse-mean'][-1]

In [9]:
# study = optuna.create_study(direction='minimize')  # Minimize RMSE

# # Optimize the study
# study.optimize(objective, n_trials=50)

# # Print the best hyperparameters
# print('Best trial:')
# trial = study.best_trial
# print(f'  Value (RMSE): {trial.value:.4f}')
# print('  Params: ')
# for key, value in trial.params.items():
#     print(f'    {key}: {value}')

In [10]:
#  Trial 29 finished with value: 38.89599916117894 and parameters: {'n_estimators': 1421, 'learning_rate': 0.016084331409244715, 'max_depth': 11, 'reg_alpha': 0.4985468335906513, 'reg_lambda': 0.6107207466582063, 'min_child_samples': 7, 'colsample_bytree': 0.7017284288029616, 'subsample': 0.8073887788934007, 'num_leaves': 53, 'min_split_gain': 0.10324878080643997}. Best is trial 29 with value: 38.89599916117894.

In [11]:
params = {'n_estimators': 1421,
          'learning_rate': 0.016084331409244715,
          'max_depth': 11,
          'reg_alpha': 0.4985468335906513,
          'reg_lambda': 0.6107207466582063,
          'min_child_samples': 7,
          'colsample_bytree': 0.7017284288029616,
          'subsample': 0.8073887788934007,
          'num_leaves': 53,
          'min_split_gain': 0.10324878080643997}

In [13]:
final_model = lgb.train(
    params,
    lgb_data
)

# Save the model
final_model.save_model('lightgbm_final_model.txt')

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data points in the train set: 3994318, number of used features: 31
[LightGBM] [Info] Start training from score 81.362175


In [15]:
loaded_model = lgb.Booster(model_file='lightgbm_final_model.txt')

In [16]:
y_pred = final_model.predict(test, num_iteration=final_model.best_iteration)
y_pred

array([81.02276384, 81.86893138, 82.54849313, ..., 83.35050453,
       82.4576022 , 80.96363678])

In [17]:
submission = pd.DataFrame({'Price': y_pred})
submission = submission.set_index(test_idx)
submission

,Price
id,
300000,81.022764
300001,81.868931
300002,82.548493
300003,80.578076
300004,78.959716
...,...
499995,79.776286
499996,77.618046
499997,83.350505


In [18]:
submission.to_csv('submission.csv')